#Author:Brian Erichsen Fagundes
#MSD CS 6017 - Summer - 2024
#Homework 2

Part 1: A/B Hypothesis testing

You conduct an A/B test for your website by showing A to some of your users, and B to others.

ad clicks	total views
A	500	1000
B	550	1000
It looks like B is better than A...

Perform a hypothesis test using the two-proportion z-value test See the "pooled" version from the table here statistic at both the 5% and 1% signficance levels.

Begin by formulating and writing down your null hypothesis, then compute the Z value for this trial, and its associated p-value.

Note: the Z value you compute is assumed to be distributed according to the standard normal distribution (mean = 0, variance = 1). We'll consider the "1-sided" p-value which meaures the probability of a sample having a Z value that is more extreme in only one tail (to get the 2-sided value, you'd just multiply by 2).

What can you conclude?

Note, to help you understand your results, you might want to tweak the number of clicks for b to see how that changes your results. For example, what do you expect to happen to your p value if b is clicked 560 or 570 times? This is JUST for understanding the results, and shouldn't bias your decision one way or another!

- Null hypotheses H0: The proportions of ad click for logos A and B are the same. H0 = pA = pB
- Alterntive hypotheses H1: The proportions of ad clicks for logos B is higher than for logo A. H1 = pB > pA

In [69]:
import math
from scipy.stats import norm
import statsmodels.api as sm

# Data
clicks_A = 500
views_A = 1000
clicks_B = 550
views_B = 1000

# p value for data a and data b
p_A = clicks_A / views_A
p_B = clicks_B / views_B

# In Binomial distribution mean = np and variance = p(1 - p)
# Var of sample proportion Var(p) = p(1 - p) / n
# Standard Error is the square root of the combined variance
# which simplified becomes sqrt of p_pool * ( 1 - p_pool) * (1 / popA + 1/ popB))
# z score is Z = (pB - pA) / SE

# pooled proportion
p_pooled = (clicks_A + clicks_B) / (views_A + views_B)

# Standard Error
SE = math.sqrt(p_pooled * (1 - p_pooled) * (1/views_A + 1/views_B))

Z = (p_B - p_A) / SE

# Cumulative Distribution Function - prob that  normally distributed
# random var is less or equal than given value

# One sided p-value // sees right of Z which is chance of
# observing an extreme value if H0 is true
p_value_one_sided = 1 - norm.cdf(Z)
p_value_two_sided = p_value_one_sided * 2

print(f"Pooled Proportion: {p_pooled}")
print(f"Stadandard Error: {SE}")
print(f"Z Value: {Z}")
print(f"One-sided p-value: {p_value_one_sided}")
print(f"Two-sided p-value: {p_value_two_sided}")

# Comfirming the results
count = [clicks_B, clicks_A]
nobs = [views_B, views_A]
z_stat, p_value = sm.stats.proportions_ztest(count, nobs, alternative='larger')
print(f"Z value (statsmodels): {z_stat}")
print(f"One-sided p-value (statsmodels): {p_value}")

# Significance levels
alpha_5 = 0.05
alpha_1 = 0.01

# Conclusions
if p_value < alpha_5:
    print("Reject the null hypothesis at the 5% significance level.")
else:
    print("Fail to reject the null hypothesis at the 5% significance level")



Pooled Proportion: 0.525
Stadandard Error: 0.022332711434127295
Z Value: 2.238868314198227
One-sided p-value: 0.012582242850159187
Two-sided p-value: 0.025164485700318373
Z value (statsmodels): 2.238868314198227
One-sided p-value (statsmodels): 0.0125822428501592
Reject the null hypothesis at the 5% significance level.


What can you conclude?

Reject the null hypothesis at the 5% significance level.


Part 2: Regression of real estate data
Adapted from datasciencecourse.net

For this problem, you will analyze SLC real estate data. The dataset contains multiple listing service (MLS) real estate transactions for houses sold in 2015-16 in zip code 84103 (SLC avenues neighborhod Google Map). We are primarily interested in regressing the SoldPrice on the house attributes (property size, house size, number of bedrooms, etc...).

Task 1: Import the data
Use the pandas.read_csv() function to import the dataset. The data is contained in two files: realEstate1.csv and realEstate2.csv. After you import these files separately, concatenate them into one big dataframe. This pandas dataframe will be used for data exploration and linear regression.

In [70]:
import pandas as pd

# load file into objects
realEstate1 = pd.read_csv('realEstate1.csv')
realEstate2 = pd.read_csv('realEstate2.csv')

# concatenate the files data
realEstate = pd.concat(
    [realEstate1, realEstate2]
)

# check first entries of combined dataframe
print(realEstate.head())

# Check for missing values and data types
print(realEstate.info())
print(realEstate.isnull().sum())



    ListNo    Access  AdditInfo  Acres             ListAgt  \
0  1397571  Concrete        NaN   0.29          Liz Slager   
1  1394021   Asphalt        NaN   0.01      Angela Carlson   
2  1393604       NaN        NaN   0.11     Rachel Kitterer   
3  1393602  Concrete        NaN   0.14     Rachel Kitterer   
4  1393829   Asphalt        NaN   0.01  T. Carter Maudsley   

                 AirType                                          Amenities  \
0  Central Air; Electric  See Remarks; Cable Tv Available; Cable Tv Wire...   
1  Central Air; Electric          Cable Tv Available; Electric Dryer Hookup   
2  Central Air; Electric              Cable Tv Wired; Electric Dryer Hookup   
3  Central Air; Electric              Cable Tv Wired; Electric Dryer Hookup   
4  Central Air; Electric                                     Cable Tv Wired   

   Area  BackDim                            Basemnt  ...  WinEvp1 WinEvp2  \
0   101      0.0  Daylight; Entrance; Full; Walkout  ...        0       0  

Task 2: Clean the data
There are 206 different variables associated with each of the 348 houses in this dataset. Skim them and try to get a rough understanding of what information this dataset contains. If you've never seen a real estate listing before, you might take a look at one on this website to get a better sense of the meanings of the column headers in the dataset.

Only keep houses with List Price between 200,000 and 1,000,000 dollars. This is an arbitrary choice and we realize that some of you are high rollers, but for our purposes we'll consider the others as outliers. Also, this data is ~10 years old. There DID use to be houses available for under $200k!

Remove columns that you don't think contribute to the value of the house. This is a personal decision - what attributes of a house are important to you? You should at least keep the following variables since the questions below will use them: ['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']

Check the datatypes and convert any numbers that were read as strings to numerical values. (Hint: You can use str.replace() to work with strings.) If there are any categorical values you're interested in, then you should convert them to numerical values as we saw in the notes (the get_dummies function may help). In particular, convert 'TotSqf' to an integer and add a column titled Prop_Type_num that is 0 if i-th listing is a condo or townhouse, or 1 if i-th listing is a single family house

Remove the listings with erroneous 'Longitude' (one has Longitude = 0) and 'Taxes' values (at least two have unreasonably large values).

In [71]:
# Filter listing by price between 200,000 and 1 M

realEstate = realEstate[(realEstate['LstPrice'] >= 200000) & (realEstate['LstPrice'] <= 1000000)]

# print(realEstate.columns)

# Keep only relevent columns
columns_to_keep = ['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']
realEstate = realEstate[columns_to_keep]
# print(realEstate.columns)

# Convert any numbers read as string to numerical value
realEstate['TotSqf'] = realEstate['TotSqf'].str.replace(',', '').astype(int)
# realEstate['Taxes'] = realEstate['Taxes'].str.replace(',', '').astype(float)
# realEstate['LstPrice'] = realEstate['LstPrice'].str.replace(',', '').astype(float)
# realEstate['SoldPrice'] = realEstate['SoldPrice'].str.replace(',', '').astype(float)

# 0 for condo and 1 for family house
realEstate['Prop_Type_num'] = realEstate['PropType'].apply(lambda x: 1 if 'Single Family' in x else 0)

# Remove listing with erronous longitude = 0
realEstate = realEstate[realEstate['Longitude'] != 0]

# Remove unreasonably large taxes values
realEstate = realEstate[realEstate['Taxes'] <= 20000]

# Display the cleaned dataframe
display(realEstate.head())

,Acres,Deck,GaragCap,Latitude,Longitude,LstPrice,Patio,PkgSpacs,PropType,SoldPrice,Taxes,TotBed,TotBth,TotSqf,YearBlt,Prop_Type_num
0,0.29,1,3,40.779524,-111.893542,725000,1,0,Single Family,752000,7244,4.0,5.0,5415,1998,1
1,0.01,0,0,40.773243,-111.883291,205000,0,0,Condo,205000,1431,2.0,1.0,924,1965,0
2,0.11,0,0,40.773972,-111.867470,390000,1,0,Single Family,390000,1816,3.0,1.0,1702,1940,1
3,0.14,0,1,40.783009,-111.872100,445000,1,3,Single Family,443000,3260,3.0,2.0,1756,1940,1
5,0.01,0,2,40.771693,-111.886951,339000,0,2,Condo,334000,2141,3.0,2.0,1368,1976,0
